In [ ]:
import os

os.makedirs("data/17flowers", exist_ok=True)
os.makedirs("data/102flowers", exist_ok=True)

In [ ]:
!wget -P data/17flowers https://thor.robots.ox.ac.uk/datasets/flowers-17/17flowers.tgz
!wget -P data/17flowers https://thor.robots.ox.ac.uk/datasets/flowers-17/datasplits.mat
!wget -P data/17flowers https://thor.robots.ox.ac.uk/datasets/flowers-17/trimaps.tgz

In [ ]:
!wget -P data/102flowers https://thor.robots.ox.ac.uk/datasets/flowers-102/102flowers.tgz
!wget -P data/102flowers https://thor.robots.ox.ac.uk/datasets/flowers-102/imagelabels.mat
!wget -P data/102flowers https://thor.robots.ox.ac.uk/datasets/flowers-102/setid.mat

In [ ]:
!tar -xzf data/17flowers/17flowers.tgz -C data/17flowers

In [ ]:
!tar -xzf data/102flowers/102flowers.tgz -C data/102flowers

In [ ]:
!rm data/102flowers/102flowers.tgz
!rm data/17flowers/17flowers.tgz

## Color constancy dataset using fc4

In [ ]:
import os

os.makedirs("data/cc/17flowers", exist_ok=True)
os.makedirs("data/cc/102flowers", exist_ok=True)

## 17flowers

In [1]:
%cd thirdparty/fc4-python3

/media/shiohiyoko/sandisk/IMLEX/DL/DL_CC_Classification/thirdparty/fc4-python3


In [ ]:
import subprocess
import sys
import cv2
import os

from fcn import FCN
from config import *
from utils import get_session
from data_provider import load_data
import utils
from datasets import get_image_pack_fn
from data_provider import ImageRecord
from PIL import Image
import numpy as np


data_path = '../../data/17flowers/jpg/files.txt'

f = open(data_path, 'r')
img_name = f.read().splitlines()
f.close()


with get_session() as sess:
    fcn = FCN(sess=sess, name='../../pretrained_colorchecker/colorchecker_fold1and2.ckpt')
    fcn.load_absolute('../../pretrained_colorchecker/colorchecker_fold1and2.ckpt')
    
    for img in img_name:
        path = 'data/17flowers/jpg/'+img

        base = os.path.dirname(os.path.abspath('./../')) # 実行ファイルのディレクトリ名
        target_path = os.path.join(base, path) # パスの連結
        # img = Image.open(target_path)
        # img = np.asarray(img)
        img = cv2.imread(target_path)
        print('image name ', target_path)
        # reverse gamma correction for sRGB
        img = (img / 255.0) ** 2.2 * 65536
        images = [img]
        fcn.test_external(images=images, fns=[target_path])

change the directory 

In [4]:
!mv cc_outputs/* ../../data/cc/17flowers/
!rm -rf cc_outputs

mv: cannot stat 'cc_outputs/*': No such file or directory


not this one

/media/shiohiyoko/sandisk/IMLEX/DL/DL_CC_Classification/data/17flowers/jpg/image_0002.jpg

In [ ]:
!python3 ./thirdparty/fc4-python3/fc4.py test ./pretrained_colorchecker/colorchecker_fold1and2.ckpt -1 ./data/17flowers ./data/cc/17flowers